In [1]:
# Cell 1: Install Libraries
%pip install pandas faiss-cpu sentence-transformers torch transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [2]:
# Cell 2: Imports and Load Data
import pandas as pd
import numpy as np
import faiss
import torch
import logging
import time
import json
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Configure logging for this part
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

# --- Configuration ---
DATA_FILE = "myscheme_schemes_data_FINAL.json"
# --- ---

logging.info("Loading data...")
try:
    # Load the JSON data into a pandas DataFrame
    df_schemes = pd.read_json(DATA_FILE, orient='records')

    # Basic Data Exploration
    logging.info(f"Data loaded successfully from {DATA_FILE}")
    print(f"DataFrame shape: {df_schemes.shape}")
    print("\nColumns:", df_schemes.columns.tolist())
    print("\nFirst few rows:")
    # Display head with potential truncation for long text fields
    with pd.option_context('display.max_colwidth', 70): # Limit column width for display
        print(df_schemes.head())
    print("\nChecking for missing values:")
    print(df_schemes.isnull().sum())

except FileNotFoundError:
    logging.error(f"Error: Data file not found at {DATA_FILE}")
    print(f"Error: Could not find the data file '{DATA_FILE}'. Please ensure it's in the correct directory.")
    # Optionally raise error to stop execution
    # raise
except Exception as e:
    logging.error(f"Error loading or processing data: {e}", exc_info=True)
    print(f"An error occurred while loading the data: {e}")
    # raise

DataFrame shape: (100, 10)

Columns: ['Source URL', 'Scheme Name', 'Ministry', 'Description', 'Category', 'Eligibility', 'Benefits', 'Application Process', 'Documents', 'Last Updated']

First few rows:
                                                      Source URL  \
0                 https://www.myscheme.gov.in/schemes/fadsp1012e   
1                   https://www.myscheme.gov.in/schemes/icmr-pdf   
2                     https://www.myscheme.gov.in/schemes/tkgthe   
3                    https://www.myscheme.gov.in/schemes/skerala   
4  https://www.myscheme.gov.in/schemes/sgassobcaniphsaislecxixii   

                                                             Scheme Name  \
0  Financial Assistance To Disabled Students Pursuing (10th, 11th, 12...   
1                                         ICMR- Post Doctoral Fellowship   
2                     Tool Kit Grant for Traditional Handicrafts Experts   
3                                                        Snehasanthwanam   
4  Scheme

In [ ]:
# Cell 3: Data Preprocessing and Chunking (Revised Concept)
import logging
import time
import json
import os
import pandas as pd # Assuming df_schemes is a pandas DataFrame

# Hypothetical Text Splitter Function (You'd need to implement or import this)
# from your_splitter_library import RecursiveCharacterTextSplitter
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=70)

def hypothetical_split_text(text, chunk_size=700, chunk_overlap=70):
    # *** This is a placeholder - Implement actual splitting logic ***
    # Example: Simple splitting for illustration, NOT production quality.
    # Replace with RecursiveCharacterTextSplitter or similar.
    if len(text) <= chunk_size:
        return [text]
    else:
        parts = []
        start = 0
        while start < len(text):
            end = start + chunk_size
            parts.append(text[start:end])
            start += chunk_size - chunk_overlap # Move start index back by overlap
            if start < 0: start = 0 # Avoid negative index if overlap > size
        # Filter out potential empty strings from splitting
        return [part for part in parts if part.strip()]


# --- Configuration ---
MIN_CHUNK_LENGTH = 15 # Reduced minimum length
DETAIL_COLUMNS = ["Eligibility", "Benefits", "Application Process", "Documents"]
MAX_FIELD_LENGTH_BEFORE_SPLIT = 300 # Example threshold
CHUNK_SIZE_FOR_SPLIT = 700 # Example target size for split parts
CHUNK_OVERLAP_FOR_SPLIT = 70 # Example overlap

logging.info("Starting data preprocessing and chunking (v2)...")

# --- Load DataFrame (Assuming df_schemes is loaded previously) ---
# Example: df_schemes = pd.read_json("myscheme_schemes_data_FINAL.json")
# Or loaded from previous cell output

chunks = []
chunk_metadata = []

# Iterate through each scheme
for index, row in df_schemes.iterrows():
    scheme_name = row.get('Scheme Name', 'Unknown Scheme')
    source_url = row.get('Source URL', 'N/A')
    ministry = row.get('Ministry', 'N/A')
    description = row.get('Description', '') # Use empty string if missing
    category = row.get('Category', '')

    # --- Chunk 1: Scheme Overview ---
    # (Keep this mostly as before, maybe check description length too)
    overview_text = (
        f"Scheme Name: {scheme_name}\n"
        f"Ministry or Department: {ministry}\n"
        f"Description: {description}\n" # Ensure description is not None
        f"Relevant Categories/Tags: {category}" # Ensure category is not None
    )
    overview_text_cleaned = ' '.join(overview_text.split())
    if len(overview_text_cleaned) > MIN_CHUNK_LENGTH: # Basic check
        chunks.append(overview_text_cleaned)
        chunk_metadata.append({
            "scheme_name": scheme_name, "source_url": source_url,
            "field": "Overview", "original_index": index
        })

    # --- Process Detail Fields ---
    for col_name in DETAIL_COLUMNS:
        detail_text = row.get(col_name)
        original_field_content = "" # Store original content before splitting

        is_valid_text = False
        if isinstance(detail_text, str):
            detail_text = detail_text.strip()
            if detail_text and not detail_text.startswith("N/A") and len(detail_text) >= MIN_CHUNK_LENGTH:
                is_valid_text = True
                original_field_content = detail_text # Store the valid text

        if is_valid_text:
            # *** NEW: Check if splitting is needed ***
            if len(original_field_content) > MAX_FIELD_LENGTH_BEFORE_SPLIT:
                logging.debug(f"Splitting long field '{col_name}' for scheme '{scheme_name[:30]}...'")
                # Use the text splitter
                split_parts = hypothetical_split_text(
                    original_field_content,
                    chunk_size=CHUNK_SIZE_FOR_SPLIT,
                    chunk_overlap=CHUNK_OVERLAP_FOR_SPLIT
                )

                for i, part in enumerate(split_parts):
                    part = part.strip()
                    if len(part) >= MIN_CHUNK_LENGTH: # Check length of split part
                         chunk_text = f"Scheme: {scheme_name} | {col_name} (Part {i+1}): {part}"
                         chunks.append(chunk_text)
                         chunk_metadata.append({
                             "scheme_name": scheme_name, "source_url": source_url,
                             "field": col_name, "original_index": index, "part_num": i+1
                         })
                    else:
                         logging.debug(f"Skipping short split part {i+1} for '{col_name}' in '{scheme_name[:30]}...'")

            else:
                # Field is short enough, create a single chunk as before
                chunk_text = f"Scheme: {scheme_name} | {col_name}: {original_field_content}"
                chunks.append(chunk_text)
                chunk_metadata.append({
                    "scheme_name": scheme_name, "source_url": source_url,
                    "field": col_name, "original_index": index
                })
        else:
             logging.debug(f"Skipping chunk creation for scheme '{scheme_name[:30]}...' field '{col_name}' due to invalid/short content.")


# --- Verification and Summary ---
logging.info(f"Chunking complete. Generated {len(chunks)} chunks from {len(df_schemes)} schemes.")
# (Rest of the verification logic from your original Cell 3)
# ...

if len(chunks) != len(chunk_metadata):
     logging.critical("CRITICAL ERROR: Mismatch between chunks count ({}) and metadata count ({})!".format(len(chunks), len(chunk_metadata)))
     print("CRITICAL ERROR: Number of chunks and metadata entries do not match!")


Total text chunks created: 499
Total metadata entries created: 499

--- Example Chunk 1 (Overview) ---
Scheme Name: Financial Assistance To Disabled Students Pursuing (10th, 11th, 12th Equivalent Exams) Ministry or Department: Kerala Description: The scheme “Financial Assistance to Disabled Students Pursuing (10th, 11th, 12th Equivalent Exams)” was launched by the Department of Social Justice, Government of Kerala. Relevant Categories/Tags: APL, BPL, Disabled, Financial Assistance, PwD, Student

--- Example Metadata 1 (Overview) ---
{'scheme_name': 'Financial Assistance To Disabled Students Pursuing (10th, 11th, 12th Equivalent Exams)', 'source_url': 'https://www.myscheme.gov.in/schemes/fadsp1012e', 'field': 'Overview', 'original_index': 0}

--- Example Chunk 2 (First Detail Field Found) ---
Scheme: Financial Assistance To Disabled Students Pursuing (10th, 11th, 12th Equivalent Exams) | Eligibility: The applicant should be a resident of Kerala State. The differently abled students wit

In [11]:
# Cell 4: Initialize Embedding Model (Re-run this cell)

logging.info("Re-initializing the sentence embedding model...") # Changed log message slightly

# --- Configuration ---
EMBEDDING_MODEL_NAME = "BAAI/bge-small-en-v1.5"
# --- ---

# Determine device
if torch.cuda.is_available():
    device = "cuda"
    logging.info("Detected CUDA-enabled GPU. Will use GPU for embeddings.")
else:
    device = "cpu"
    logging.info("No CUDA-enabled GPU detected. Using CPU for embeddings.")

# Load the Sentence Transformer model
try:
    start_load_time = time.time()
    # Instantiate the model, sending it to the chosen device
    embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device=device)
    end_load_time = time.time()

    # Confirm successful loading
    logging.info(f"Embedding model '{EMBEDDING_MODEL_NAME}' loaded successfully to '{device}'.")
    print(f"Embedding model loaded/re-loaded in {end_load_time - start_load_time:.2f} seconds.")

except Exception as e:
    logging.error(f"Failed to load embedding model '{EMBEDDING_MODEL_NAME}': {e}", exc_info=True)
    print(f"Error: Could not load the embedding model. Please check the model name ('{EMBEDDING_MODEL_NAME}'), your internet connection, and library installations.")
    # raise e # Halt execution if needed

Embedding model loaded/re-loaded in 1.66 seconds.


In [5]:
# Cell 5: Generate Embeddings

logging.info("Generating embeddings for the text chunks...")

# Ensure the 'chunks' list from Step 3 exists and is not empty
if 'chunks' not in locals() or not chunks:
    logging.error("Chunk list ('chunks') is missing or empty. Cannot generate embeddings. Please re-run Step 3.")
    print("Error: Text chunks not found. Please ensure Step 3 (Data Preprocessing and Chunking) ran successfully.")
    # Optionally, raise an error to halt execution if chunks are missing
    # raise ValueError("Cannot generate embeddings without text chunks.")
else:
    try:
        logging.info(f"Starting embedding generation for {len(chunks)} chunks using '{EMBEDDING_MODEL_NAME}' on device '{device}'...")
        start_embed_time = time.time()

        # Use the encode method of the SentenceTransformer model
        # convert_to_numpy=True ensures the output is a NumPy array suitable for FAISS
        # show_progress_bar=True provides visual feedback during the process
        chunk_embeddings = embedding_model.encode(
            chunks,
            show_progress_bar=True,
            convert_to_numpy=True
        )
        end_embed_time = time.time()

        # --- Verification ---
        # Get the expected dimension size from the model
        embedding_dimension = embedding_model.get_sentence_embedding_dimension()
        actual_shape = chunk_embeddings.shape

        logging.info(f"Embeddings generated successfully in {end_embed_time - start_embed_time:.2f} seconds.")
        print(f"\nEmbeddings generated for {actual_shape[0]} chunks.")
        print(f"Shape of embeddings array: {actual_shape}") # Expected: (number_of_chunks, embedding_dimension)

        # Verify dimensions match expectations
        if actual_shape[0] == len(chunks) and actual_shape[1] == embedding_dimension:
            print(f"Embeddings dimension ({actual_shape[1]}D) matches model's expected dimension.")
        else:
            logging.warning(f"Embeddings shape mismatch! Expected ({len(chunks)}, {embedding_dimension}), Got {actual_shape}")
            print(f"Warning: Embeddings array shape {actual_shape} differs from expected ({len(chunks)}, {embedding_dimension}).")

    except Exception as e:
        logging.error(f"An error occurred during embedding generation: {e}", exc_info=True)
        print(f"An error occurred while generating embeddings: {e}")
        # Depending on the error, might need to handle or halt
        # raise e

Batches:   0%|          | 0/16 [00:00<?, ?it/s]


Embeddings generated for 499 chunks.
Shape of embeddings array: (499, 384)
Embeddings dimension (384D) matches model's expected dimension.


In [6]:
# Cell 6: Build and Save FAISS Index

logging.info("Building the FAISS index...")

# Ensure the embeddings NumPy array exists from Step 5
if 'chunk_embeddings' not in locals() or not isinstance(chunk_embeddings, np.ndarray):
    logging.error("Embeddings array ('chunk_embeddings') not found. Cannot build index. Please re-run Step 5.")
    print("Error: Embeddings variable is missing. Please ensure Step 5 ran correctly.")
    # Optionally raise error
    # raise NameError("chunk_embeddings not defined")
else:
    try:
        # 1. Get the dimensionality (d) from the embeddings array
        d = chunk_embeddings.shape[1]
        n_vectors = chunk_embeddings.shape[0]
        logging.info(f"Embeddings shape: ({n_vectors}, {d})")

        # 2. Normalize embeddings for IndexFlatIP (to compute Cosine Similarity)
        # FAISS works with float32
        if chunk_embeddings.dtype != np.float32:
            logging.info(f"Converting embeddings dtype from {chunk_embeddings.dtype} to float32.")
            chunk_embeddings = chunk_embeddings.astype(np.float32)

        logging.info("Normalizing embeddings using faiss.normalize_L2...")
        faiss.normalize_L2(chunk_embeddings)
        logging.info("Embeddings normalized successfully.")

        # 3. Create a FAISS index - IndexFlatIP supports Inner Product search
        # For normalized vectors, max Inner Product corresponds to max Cosine Similarity
        index = faiss.IndexFlatIP(d)
        logging.info(f"FAISS index created: Type=IndexFlatIP, Dimension={d}")

        # 4. Add the normalized vectors to the index
        index.add(chunk_embeddings)
        logging.info(f"Added {index.ntotal} vectors to the index.")

        # --- Verification ---
        if index.ntotal == n_vectors:
            print(f"\nFAISS index built successfully with {index.ntotal} vectors.")
            print(f"Index is trained: {index.is_trained}") # IndexFlatIP requires no training
        else:
            logging.error(f"Index count mismatch! Expected {n_vectors}, found {index.ntotal}")
            print(f"\nError: Number of vectors in index ({index.ntotal}) does not match input ({n_vectors}).")


        # --- Save Index and Associated Data ---
        # Define filenames for persistence
        FAISS_INDEX_FILE = "my_scheme_faiss.index"
        METADATA_FILE = "my_scheme_metadata.json"
        CHUNKS_FILE = "my_scheme_chunks.json"

        # Save the FAISS index
        logging.info(f"Saving FAISS index to '{FAISS_INDEX_FILE}'...")
        faiss.write_index(index, FAISS_INDEX_FILE)
        logging.info("FAISS index saved.")

        # Save the metadata (list of dictionaries)
        logging.info(f"Saving chunk metadata to '{METADATA_FILE}'...")
        # Ensure 'chunk_metadata' list exists from Step 3
        if 'chunk_metadata' in locals() and chunk_metadata:
            with open(METADATA_FILE, 'w', encoding='utf-8') as f_meta:
                # Use json module to dump the list of dicts
                json.dump(chunk_metadata, f_meta, ensure_ascii=False, indent=2)
            logging.info("Chunk metadata saved.")
        else:
            logging.warning("Chunk metadata not found or empty, cannot save metadata file.")

        # Save the actual text chunks (list of strings)
        logging.info(f"Saving text chunks to '{CHUNKS_FILE}'...")
        # Ensure 'chunks' list exists from Step 3
        if 'chunks' in locals() and chunks:
             with open(CHUNKS_FILE, 'w', encoding='utf-8') as f_chunks:
                  json.dump(chunks, f_chunks, ensure_ascii=False, indent=2) # Save list of strings
             logging.info("Text chunks saved.")
        else:
             logging.warning("Chunks list not found or empty, cannot save chunks file.")


        print(f"\nIndex and associated data saved:")
        print(f"- Index: {FAISS_INDEX_FILE}")
        print(f"- Metadata: {METADATA_FILE}")
        print(f"- Chunks: {CHUNKS_FILE}")


    except Exception as e:
        logging.error(f"An error occurred during FAISS index building or saving: {e}", exc_info=True)
        print(f"An error occurred during index creation/saving: {e}")
        # raise e


FAISS index built successfully with 499 vectors.
Index is trained: True

Index and associated data saved:
- Index: my_scheme_faiss.index
- Metadata: my_scheme_metadata.json
- Chunks: my_scheme_chunks.json


In [16]:
# Cell 7: Initialize Generator Model (Re-run this cell)

logging.info("Re-initializing the Generator (Seq2Seq) Language Model...") # Modified log message

# --- Configuration ---
GENERATOR_MODEL_NAME = "google/flan-t5-base"
# --- ---

# Re-determine device
if torch.cuda.is_available():
    device = "cuda"
    logging.info(f"GPU ({torch.cuda.get_device_name(0)}) found. Attempting to load generator model to GPU.")
else:
    device = "cpu"
    logging.info("Using CPU for generator model.")

try:
    start_load_time = time.time()

    # Load the tokenizer
    generator_tokenizer = AutoTokenizer.from_pretrained(GENERATOR_MODEL_NAME)
    logging.info(f"Tokenizer for '{GENERATOR_MODEL_NAME}' loaded.")

    # Load the model
    try:
         generator_model = AutoModelForSeq2SeqLM.from_pretrained(GENERATOR_MODEL_NAME, device_map="auto")
         logging.info("Generator model loaded with device_map='auto'.")
    except Exception as auto_map_error:
         logging.warning(f"Loading with device_map='auto' failed ({auto_map_error}). Attempting manual placement on '{device}'...")
         generator_model = AutoModelForSeq2SeqLM.from_pretrained(GENERATOR_MODEL_NAME)
         generator_model.to(device) # Move the model to the determined device

    end_load_time = time.time()

    final_device = next(generator_model.parameters()).device
    logging.info(f"Generator model '{GENERATOR_MODEL_NAME}' is ready on device: {final_device}.")
    print(f"\nGenerator model loaded/re-loaded in {end_load_time - start_load_time:.2f} seconds.")
    print(f"Model is actually on device: {final_device}")

    generator_model.eval()
    logging.info("Generator model set to evaluation mode.")

except Exception as e:
    logging.error(f"Failed to load generator model or tokenizer '{GENERATOR_MODEL_NAME}': {e}", exc_info=True)
    print(f"Error: Could not load the generator model/tokenizer. Check model name, connection, and resources (RAM/VRAM).")
    # raise e # Halt execution if needed


Generator model loaded/re-loaded in 1.03 seconds.
Model is actually on device: cuda:0


In [9]:
# Cell 8a: Load Index and Data from Files

logging.info("Attempting to load FAISS index, chunks, and metadata from files...")

# --- Configuration (Filenames from Step 6) ---
FAISS_INDEX_FILE = "my_scheme_faiss.index"
METADATA_FILE = "my_scheme_metadata.json"
CHUNKS_FILE = "my_scheme_chunks.json"
# --- ---

# Flag to track loading success
loaded_successfully = True

try:
    # Load FAISS index
    logging.info(f"Loading FAISS index from {FAISS_INDEX_FILE}...")
    index = faiss.read_index(FAISS_INDEX_FILE)
    logging.info(f"FAISS index loaded successfully with {index.ntotal} vectors.")
    print(f"FAISS index loaded (Vectors: {index.ntotal}).")

    # Load metadata
    logging.info(f"Loading chunk metadata from {METADATA_FILE}...")
    with open(METADATA_FILE, 'r', encoding='utf-8') as f_meta:
        chunk_metadata = json.load(f_meta)
    logging.info(f"Chunk metadata loaded successfully ({len(chunk_metadata)} entries).")
    print(f"Metadata loaded ({len(chunk_metadata)} entries).")

    # Load chunks
    logging.info(f"Loading text chunks from {CHUNKS_FILE}...")
    with open(CHUNKS_FILE, 'r', encoding='utf-8') as f_chunks:
        chunks = json.load(f_chunks)
    logging.info(f"Text chunks loaded successfully ({len(chunks)} entries).")
    print(f"Chunks loaded ({len(chunks)} entries).")

    # Verify counts match
    if not (index.ntotal == len(chunk_metadata) == len(chunks)):
         logging.warning("Mismatch in loaded counts! Index: {}, Metadata: {}, Chunks: {}".format(
              index.ntotal, len(chunk_metadata), len(chunks)
         ))
         print("Warning: Counts of loaded items (index vectors, metadata, chunks) do not match!")
         # Decide if this is critical - might still work but indicates an issue.
    else:
         print("Loaded data counts verified.")

    # Check if embedding_model is loaded (should be from Cell 4)
    if 'embedding_model' not in locals():
         logging.error("Embedding model ('embedding_model') not found in environment.")
         print("Error: Embedding model is not loaded. Please re-run Cell 4.")
         loaded_successfully = False # Mark as failed
         # raise NameError("embedding_model not loaded") # Halt if critical

    # Check if generator_model is loaded (should be from Cell 7)
    if 'generator_model' not in locals() or 'generator_tokenizer' not in locals():
         logging.warning("Generator model/tokenizer not found in environment. Will need Cell 7 to be run before generation.")
         # Don't mark as failed yet, only needed for generation step later
         print("Note: Generator model/tokenizer not loaded (needed for Step 9/10).")


except FileNotFoundError as fnf_error:
     logging.error(f"Error loading files: {fnf_error}. Cannot proceed.")
     print(f"\nError: One or more required files not found ({FAISS_INDEX_FILE}, {METADATA_FILE}, {CHUNKS_FILE}).")
     print("Please ensure Step 6 (Build and Save FAISS Index) ran successfully and saved the files in the current directory.")
     loaded_successfully = False # Mark as failed
     # raise # Stop execution
except Exception as e:
     logging.error(f"An error occurred loading index/data from files: {e}", exc_info=True)
     print(f"\nAn error occurred while loading saved index/data: {e}")
     loaded_successfully = False # Mark as failed
     # raise

if loaded_successfully:
    print("\nIndex, metadata, and chunks loaded successfully.")
else:
    print("\nLoading failed. Cannot proceed with retrieval test reliably.")

FAISS index loaded (Vectors: 499).
Metadata loaded (499 entries).
Chunks loaded (499 entries).
Loaded data counts verified.

Index, metadata, and chunks loaded successfully.


In [14]:
# Cell 8 (Explicit Debug Check): Define Retrieval Function

logging.info("Defining the retrieval function...")

# --- Function Definition (same as before) ---
def retrieve_relevant_chunks(query: str,
                              model: SentenceTransformer,
                              faiss_index: faiss.Index,
                              chunk_list: list,
                              metadata_list: list,
                              top_k: int = 5):
    if not query:
        logging.warning("Received empty query for retrieval.")
        return []
    logging.info(f"Retrieving top {top_k} relevant chunks for query: \"{query[:100]}...\"")
    try:
        t_embed_start = time.time()
        query_embedding = model.encode([query], convert_to_numpy=True, show_progress_bar=False)
        query_embedding = query_embedding.astype(np.float32)
        faiss.normalize_L2(query_embedding)
        t_embed_end = time.time()
        logging.debug(f"Query embedding generated and normalized in {t_embed_end - t_embed_start:.4f} sec. Shape: {query_embedding.shape}")
        t_search_start = time.time()
        scores, indices = faiss_index.search(query_embedding, top_k)
        t_search_end = time.time()
        logging.debug(f"FAISS search completed in {t_search_end - t_search_start:.4f} sec.")
        results = []
        if indices.size > 0 and scores.size > 0:
            retrieved_indices = indices[0]
            retrieved_scores = scores[0]
            logging.debug(f"Retrieved indices: {retrieved_indices}")
            logging.debug(f"Retrieved scores: {retrieved_scores}")
            for i, idx in enumerate(retrieved_indices):
                if idx != -1:
                     if 0 <= idx < len(chunk_list) and 0 <= idx < len(metadata_list):
                          results.append({
                              "chunk_text": chunk_list[idx],
                              "metadata": metadata_list[idx],
                              "score": float(retrieved_scores[i])
                          })
                          logging.debug(f"  -> Added chunk index {idx} (Score: {retrieved_scores[i]:.4f})")
                     else:
                          logging.warning(f"Retrieved index {idx} is out of bounds for chunk/metadata lists (Size: {len(chunk_list)}). Skipping.")
                else:
                     logging.debug(f"Retrieved index was -1 at position {i}.")
        logging.info(f"Retrieved {len(results)} chunks for the query.")
        results.sort(key=lambda x: x['score'], reverse=True)
        return results
    except Exception as e:
        logging.error(f"An error occurred during retrieval for query \"{query}\": {e}", exc_info=True)
        return []
# --- End of Function Definition ---


# --- Explicit Debugging Check ---
print("\n" + "="*10 + " Checking Variable Existence Before Test " + "="*10)
required_vars_to_check = ['embedding_model', 'index', 'chunks', 'chunk_metadata']
are_all_vars_defined = True
variables_status = {}

# Check both local and global scopes explicitly
current_locals = locals()
current_globals = globals()

for var_name in required_vars_to_check:
    found_in_locals = var_name in current_locals
    found_in_globals = var_name in current_globals

    if found_in_locals:
        status = f"FOUND in locals(). Type: {type(current_locals[var_name])}"
        variables_status[var_name] = True
    elif found_in_globals:
        status = f"FOUND in globals(). Type: {type(current_globals[var_name])}"
        variables_status[var_name] = True
    else:
        status = "NOT FOUND in locals() or globals()."
        variables_status[var_name] = False
        are_all_vars_defined = False # Mark that at least one is missing

    print(f"Variable '{var_name}': {status}")

print(f"\nOverall check result: are_all_vars_defined = {are_all_vars_defined}")
print("="*50 + "\n")
# --- End Explicit Debugging Check ---


# --- Quick Test of the Retrieval Function ---
# Use the flag determined by the explicit check above
if are_all_vars_defined:
    print("-" * 50)
    print("--- Testing Retrieval Function ---")
    test_query = "What schemes are available for farmers in Maharashtra?"
    print(f"Test Query: \"{test_query}\"")
    # Directly use the variables, assuming the check passed
    retrieved_data = retrieve_relevant_chunks(test_query, embedding_model, index, chunks, chunk_metadata, top_k=3)

    if retrieved_data:
        print(f"\nSuccessfully retrieved {len(retrieved_data)} results:")
        for rank, result in enumerate(retrieved_data, 1):
            print(f"\n--- Result #{rank} ---")
            print(f"  Similarity Score: {result['score']:.4f}")
            print(f"  Metadata:")
            for k, v in result['metadata'].items():
                 print(f"    {k}: {v}")
            print(f"  Chunk Text (Snippet):")
            print(f"    \"{result['chunk_text'][:300]}...\"")
    else:
        print("\nNo results were retrieved for the test query.")
    print("-" * 50)
else:
    print("\nSkipping retrieval function test because the explicit check confirmed one or more required components were missing.")
    logging.warning("Skipping retrieval test because explicit components check failed.")


========== Checking Variable Existence Before Test ==========
Variable 'embedding_model': FOUND in locals(). Type: <class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>
Variable 'index': FOUND in locals(). Type: <class 'faiss.swigfaiss_avx512.IndexFlatIP'>
Variable 'chunks': FOUND in locals(). Type: <class 'list'>
Variable 'chunk_metadata': FOUND in locals(). Type: <class 'list'>

Overall check result: are_all_vars_defined = True

--------------------------------------------------
--- Testing Retrieval Function ---
Test Query: "What schemes are available for farmers in Maharashtra?"

Successfully retrieved 3 results:

--- Result #1 ---
  Similarity Score: 0.7096
  Metadata:
    scheme_name: YSR Jala Kala
    source_url: https://www.myscheme.gov.in/schemes/ysrjk
    field: Eligibility
    original_index: 97
  Chunk Text (Snippet):
    "Scheme: YSR Jala Kala | Eligibility: The applicant should be a Farmer. The applicant should be a resident of the state of Andhra Prade

In [18]:
# Cell 9 (Attempt 3 - Remove Pre-check): Define Generation Function

logging.info("Defining the answer generation function (Attempt 3)...")

# Assuming generator_model and generator_tokenizer exist from Cell 7 execution

# --- Generation Configuration ---
MAX_ANSWER_LENGTH = 256
NUM_BEAMS = 4
NO_REPEAT_NGRAM = 2
EARLY_STOPPING = True
# --- ---

def generate_answer(query: str,
                     context_chunk_texts: list,
                     model: AutoModelForSeq2SeqLM,
                     tokenizer: AutoTokenizer):
    """
    Generates an answer using the seq2seq model based on the query and context.
    """
    # Try using the models directly - Python will raise NameError if they don't exist
    # Use global keyword if needed, though generally not required in notebooks unless defining inside another function
    # global generator_model, generator_tokenizer

    if not context_chunk_texts:
        logging.warning("generate_answer called with no context provided.")
        return "Based on the available scheme information, I could not find specific details to answer your question."

    context_string = "\n\n".join(context_chunk_texts)
    prompt_template = """Please answer the following question based *only* on the provided context information. If the context does not contain the answer, please state 'Based on the provided context, I cannot answer this question'.

Context:
\"\"\"
{context}
\"\"\"

Question: {question}

Answer:"""
    prompt = prompt_template.format(context=context_string, question=query)

    logging.info(f"Generating answer for query: '{query[:100]}...'")
    logging.debug(f"Generator Prompt (start):\n{prompt[:500]}...")

    try:
        t_token_start = time.time()
        # Use the assumed globally available 'generator_tokenizer' and 'generator_model'
        inputs = generator_tokenizer(prompt,
                                      return_tensors="pt",
                                      max_length=1024,
                                      truncation=True
                                     ).to(generator_model.device) # Use model's stored device
        t_token_end = time.time()
        logging.debug(f"Prompt tokenized in {t_token_end - t_token_start:.4f} sec.")

        t_gen_start = time.time()
        output_sequences = generator_model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=MAX_ANSWER_LENGTH,
            num_beams=NUM_BEAMS,
            early_stopping=EARLY_STOPPING,
            no_repeat_ngram_size=NO_REPEAT_NGRAM
        )
        t_gen_end = time.time()
        logging.debug(f"Output sequences generated in {t_gen_end - t_gen_start:.4f} sec.")

        t_decode_start = time.time()
        generated_text = generator_tokenizer.decode(output_sequences[0], skip_special_tokens=True)
        t_decode_end = time.time()
        logging.debug(f"Response decoded in {t_decode_end - t_decode_start:.4f} sec.")

        final_answer = generated_text.strip()
        logging.info(f"Generated Answer (final): '{final_answer[:150]}...'")
        return final_answer

    except NameError as ne:
         # Catch NameError specifically if model/tokenizer weren't actually loaded
         logging.error(f"NameError during generation: {ne}. Model or Tokenizer likely missing.", exc_info=True)
         print(f"ERROR: A required model component ({ne}) was not found. Please ensure Cell 7 was run successfully.")
         return "Error: Generation components missing."
    except Exception as e:
         logging.error(f"An error occurred during answer generation: {e}", exc_info=True)
         return "I encountered an error while trying to generate the answer."


# --- Quick Test of the Generation Function ---
print("\n" + "-" * 50)
print("--- Testing Generation Function ---")

# Check for retrieved_data existence before testing
if 'retrieved_data' in locals() and retrieved_data:
    test_context_texts = [res['chunk_text'] for res in retrieved_data]
    # Check if the test variables actually exist before calling generate_answer
    if 'generator_model' in locals() and 'generator_tokenizer' in locals():
        test_gen_query = "Who is eligible for YSR Jala Kala?"

        print(f"Test Generation Query: '{test_gen_query}'")
        print(f"Using context from {len(test_context_texts)} chunks retrieved in the previous test.")

        # Call the generation function
        generated_answer = generate_answer(test_gen_query, test_context_texts, generator_model, generator_tokenizer)

        print("\nGenerated Answer:")
        print(generated_answer)
    else:
        print("\nCannot run generation test because 'generator_model' or 'generator_tokenizer' is missing.")
        logging.error("Cannot run generation test - generator model/tokenizer missing.")

else:
    # This might happen if Cell 8's test failed or wasn't run
    print("\nSkipping generation test as no retrieved context ('retrieved_data') is available.")
    logging.warning("Skipping generation test - no context available.")
print("-" * 50)


--------------------------------------------------
--- Testing Generation Function ---
Test Generation Query: 'Who is eligible for YSR Jala Kala?'
Using context from 3 chunks retrieved in the previous test.

Generated Answer:
A Farmer.
--------------------------------------------------


In [22]:
# Cell 10 (Corrected NameError): Define Combined QA Function and Test

logging.info("Defining the main QA function...")

# Check dependencies are loaded one last time before defining the main function
required_components = [
    'embedding_model', 'index', 'chunks', 'chunk_metadata',
    'generator_model', 'generator_tokenizer',
    'retrieve_relevant_chunks', 'generate_answer'
]
# Also check for LOGGING_LEVEL which we will use for the debug check
if 'LOGGING_LEVEL' not in locals() and 'LOGGING_LEVEL' not in globals():
     # If LOGGING_LEVEL wasn't set explicitly, check the root logger's level
     if logging.getLogger().hasHandlers():
          LOGGING_LEVEL = logging.getLogger().getEffectiveLevel()
          logging.info(f"LOGGING_LEVEL variable not found, inferred level as {logging.getLevelName(LOGGING_LEVEL)} from root logger.")
     else:
          # Fallback if logging wasn't configured properly
          LOGGING_LEVEL = logging.WARNING
          logging.warning("LOGGING_LEVEL variable not found and root logger not configured, defaulting to WARNING.")


if not all(comp in locals() or comp in globals() for comp in required_components):
     logging.critical("Cannot define QA function - one or more critical components are missing!")
     print("\nError: Critical components missing. Cannot proceed. Please ensure previous cells ran successfully.")
     # raise NameError("Missing components for QA function")
else:
     def ask_scheme_question(query: str, top_k_chunks: int = 5):
          """
          Orchestrates the RAG process: retrieves context and generates an answer.
          Args:
              query (str): The user's question about government schemes.
              top_k_chunks (int): The number of relevant chunks to retrieve.
          Returns:
              str: The generated answer.
          """
          logging.info(f"Processing QA query: \"{query}\"")

          # 1. Retrieve Relevant Context Chunks
          retrieved_context = retrieve_relevant_chunks(
              query=query,
              model=embedding_model,
              faiss_index=index,
              chunk_list=chunks,
              metadata_list=chunk_metadata,
              top_k=top_k_chunks # Corrected keyword argument from previous step
          )

          if not retrieved_context:
               logging.warning("Retrieval returned no relevant chunks for the query.")
               return "I couldn't find specific information related to your question in the available scheme data."

          context_texts = [item['chunk_text'] for item in retrieved_context]

          # --- CORRECTED LINE: Use LOGGING_LEVEL ---
          if LOGGING_LEVEL <= logging.DEBUG:
               logging.debug(f"Passing {len(context_texts)} chunks to the generator.")
               for i, txt in enumerate(context_texts):
                    logging.debug(f"Context Chunk {i+1}: {txt[:100]}...")
          # --- END CORRECTION ---

          # 2. Generate Answer using Retrieved Context
          final_answer = generate_answer(
              query=query,
              context_chunk_texts=context_texts,
              model=generator_model,
              tokenizer=generator_tokenizer
          )

          # 3. Return the generated answer
          return final_answer

     # --- Test the Full QA Pipeline ---
     print("\n" + "="*50)
     print("--- Testing the Full QA System ---")

     test_queries = [
         "What schemes are available for farmers in Maharashtra?", # User's original example
         "What is the benefit amount for the PM Scholarship for RPF?", # Specific, likely answerable
         "Tell me about eligibility for Snehasanthwanam.", # Specific, likely answerable
         "Are there any schemes for building houses?", # General, might find relevant context
         "What are the documents required for the Laadli Laxmi Scheme?", # Specific, likely answerable
         "What is the MyScheme portal?" # General knowledge, unlikely in context
     ]

     for i, q in enumerate(test_queries, 1):
          print(f"\n--- Query #{i} ---")
          print(f"Q: {q}")
          answer = ask_scheme_question(q, top_k_chunks=5) # Use top 5 chunks
          print(f"A: {answer}")
          # time.sleep(1) # Optional delay

     print("\n" + "="*50)
     print("--- QA System Testing Complete ---")


--- Testing the Full QA System ---

--- Query #1 ---
Q: What schemes are available for farmers in Maharashtra?
A: Chartered Accountants

--- Query #2 ---
Q: What is the benefit amount for the PM Scholarship for RPF?
A: 2500/- per month for male students

--- Query #3 ---
Q: Tell me about eligibility for Snehasanthwanam.
A: The applicant should be a resident of Kerala State

--- Query #4 ---
Q: Are there any schemes for building houses?
A: The Atal Mission For Rejuvenation And Urban Transformation

--- Query #5 ---
Q: What are the documents required for the Laadli Laxmi Scheme?
A: The application shall be submitted within one year, from the date of attaining the age of 18 years

--- Query #6 ---
Q: What is the MyScheme portal?
A: Department of Education.

--- QA System Testing Complete ---


In [23]:
# Cell 11: Quit WebDriver

try:
    # Check if 'driver' variable exists and holds a WebDriver instance
    if 'driver' in locals() and isinstance(driver, webdriver.WebDriver):
        logging.info("Closing the Selenium WebDriver...")
        driver.quit()
        logging.info("WebDriver closed successfully.")
        print("\nBrowser window (if any) closed.")
        # Optional: remove variable from namespace
        # del driver
    elif 'driver' in locals():
         logging.warning("Variable 'driver' exists but is not a WebDriver instance or might be None.")
         print("\nWebDriver already closed or variable reassigned.")
    else:
        logging.info("WebDriver variable 'driver' not found. Assuming already closed.")
        print("\nBrowser window (if any) likely already closed or was not used in this session.")
except NameError:
     logging.info("WebDriver variable 'driver' not defined in this scope.")
     print("\nBrowser window (if any) likely already closed or was not used in this session.")
except Exception as e:
    logging.error(f"An error occurred while trying to close the WebDriver: {e}")
    print(f"\nAn error occurred closing browser: {e}")

print("\nQA System setup and testing complete.")


Browser window (if any) likely already closed or was not used in this session.

QA System setup and testing complete.


In [ ]:
#Complete code (combined all cell)
# run_scheme_qa.py
# RAG QA system using pre-processed MyScheme data.
# Assumes scraping is done and necessary RAG data files exist.

import logging
import time
import numpy as np
import faiss
import torch
import json
import os # For checking file existence

# RAG Imports
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# --- Configuration ---
# Input Files (Assumed to exist from previous indexing step)
FAISS_INDEX_FILE = "my_scheme_faiss.index"
METADATA_FILE = "my_scheme_metadata.json"
CHUNKS_FILE = "my_scheme_chunks.json"

# Model Config
EMBEDDING_MODEL_NAME = "BAAI/bge-small-en-v1.5"
GENERATOR_MODEL_NAME = "google/flan-t5-base"

# RAG Config
RAG_TOP_K = 5 # Number of chunks to retrieve for RAG context

# General Config
LOGGING_LEVEL = logging.INFO # Change to logging.DEBUG for very detailed output

# Configure Logging
logging.basicConfig(level=LOGGING_LEVEL,
                    format='%(asctime)s [%(levelname)-8s] %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')
log = logging.getLogger(__name__)

# --- RAG QA SYSTEM ---

def setup_and_run_qa_system(index_path=FAISS_INDEX_FILE,
                             metadata_path=METADATA_FILE,
                             chunks_path=CHUNKS_FILE):
    """
    Loads pre-built RAG components (index, chunks, metadata),
    loads necessary models (embedding, generator), defines QA functions,
    and runs test queries.
    """
    log.info("--- Starting RAG QA System Setup & Test ---")
    start_rag_time = time.time()

    # --- 1. Load Pre-built Index and Data ---
    log.info("Loading pre-built FAISS index and associated data...")
    required_files = [index_path, metadata_path, chunks_path]
    if not all(os.path.exists(f) for f in required_files):
        log.critical(f"One or more required RAG data files not found: {required_files}. Cannot proceed.")
        print(f"Error: Missing required input files. Please ensure '{index_path}', '{metadata_path}', and '{chunks_path}' exist from the previous processing step.")
        return False # Indicate failure

    try:
        index = faiss.read_index(index_path)
        log.info(f"FAISS index loaded from '{index_path}' ({index.ntotal} vectors).")
        with open(metadata_path, 'r', encoding='utf-8') as f:
            chunk_metadata = json.load(f)
        log.info(f"Chunk metadata loaded from '{metadata_path}' ({len(chunk_metadata)} entries).")
        with open(chunks_path, 'r', encoding='utf-8') as f:
            chunks = json.load(f)
        log.info(f"Text chunks loaded from '{chunks_path}' ({len(chunks)} entries).")

        # Verify counts match
        if not (index.ntotal == len(chunk_metadata) == len(chunks)):
             log.warning(f"Loaded data count mismatch! Index:{index.ntotal}, Meta:{len(chunk_metadata)}, Chunks:{len(chunks)}")
             print("Warning: Counts of loaded items (index vectors, metadata, chunks) do not match! Proceeding cautiously.")
             # Decide if this is critical based on use case
    except Exception as e:
        log.critical(f"Failed to load RAG data files: {e}", exc_info=True)
        print(f"Error loading required data files: {e}")
        return False

    # --- 2. Load Embedding Model (Needed for querying) ---
    log.info(f"Loading embedding model: {EMBEDDING_MODEL_NAME}...")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    log.info(f"Using device: {device} for embeddings")
    try:
        # Define embedding_model in this scope
        embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME, device=device)
        log.info("Embedding model loaded.")
    except Exception as e:
        log.critical(f"Failed to load embedding model: {e}", exc_info=True)
        print(f"Error loading embedding model: {e}")
        return False

    # --- 3. Load Generator Model (Needed for generation) ---
    log.info(f"Loading generator model: {GENERATOR_MODEL_NAME}...")
    try:
        # Define generator_tokenizer and generator_model in this scope
        generator_tokenizer = AutoTokenizer.from_pretrained(GENERATOR_MODEL_NAME)
        # Use device_map='auto' for flexible placement (GPU/CPU/multi-GPU)
        generator_model = AutoModelForSeq2SeqLM.from_pretrained(GENERATOR_MODEL_NAME, device_map="auto")
        generator_model.eval() # Set to evaluation mode
        log.info(f"Generator model ready on device(s): {generator_model.device_map if hasattr(generator_model, 'device_map') else next(generator_model.parameters()).device}")
    except Exception as e:
        log.critical(f"Failed to load generator model/tokenizer: {e}", exc_info=True)
        print(f"Error loading generator model: {e}")
        return False

    # --- 4. Define RAG Functions ---
    log.info("Defining RAG pipeline functions...")

    # Retrieval Function
    def retrieve_relevant_chunks_rag(query: str, top_k: int = RAG_TOP_K):
        # This function now uses the variables loaded/defined within setup_and_run_qa_system
        logging.debug(f"Retrieving top {top_k} for: '{query[:60]}...'")
        try:
            query_embedding = embedding_model.encode([query], convert_to_numpy=True, show_progress_bar=False)
            query_embedding = query_embedding.astype(np.float32)
            faiss.normalize_L2(query_embedding)
            scores, indices = index.search(query_embedding, top_k)
            results = []
            if indices.size > 0:
                 for i, idx in enumerate(indices[0]):
                      if idx != -1 and 0 <= idx < len(chunks):
                           results.append({"chunk_text": chunks[idx], "metadata": chunk_metadata[idx], "score": float(scores[0][i])})
            results.sort(key=lambda x: x['score'], reverse=True) # Highest score first
            log.debug(f"Retrieved {len(results)} chunks.")
            return results
        except Exception as e: log.error(f"Retrieval error: {e}", exc_info=True); return []

    # Generation Function
    def generate_answer_rag(query: str, context_chunk_texts: list):
        if not context_chunk_texts: return "I couldn't find relevant information in the provided context."
        context_string = "\n\n".join(context_chunk_texts)
        prompt_template = """Please answer the following question based *only* on the provided context information. If the context does not contain the answer, please state 'Based on the provided context, I cannot answer this question'.

Context:
---
{context}
---

Question: {question}

Answer:"""
        prompt = prompt_template.format(context=context_string, question=query)
        log.debug(f"Generator prompt (start): {prompt[:300]}...")
        try:
            inputs = generator_tokenizer(prompt, return_tensors="pt", max_length=1024, truncation=True).to(generator_model.device)
            outputs = generator_model.generate(
                **inputs,
                max_new_tokens=256, # Max tokens for the generated answer
                num_beams=4,
                early_stopping=True,
                no_repeat_ngram_size=2
                )
            answer = generator_tokenizer.decode(outputs[0], skip_special_tokens=True)
            log.debug(f"Generated answer: {answer[:100]}...")
            return answer.strip()
        except Exception as e: log.error(f"Generation error: {e}", exc_info=True); return "Error during answer generation."

    # Combined QA Function
    def ask_scheme_question_rag(query: str):
        log.info(f"QA Query: \"{query}\"")
        retrieved = retrieve_relevant_chunks_rag(query, top_k=RAG_TOP_K)
        if not retrieved:
            log.warning("No relevant chunks retrieved.")
            return "I couldn't find specific information related to your question in the available scheme data."

        context = [r['chunk_text'] for r in retrieved]
        answer = generate_answer_rag(query, context)
        return answer

    # --- 5. Test QA System ---
    log.info("--- Testing QA System ---")
    print("\n" + "="*50)
    print("--- Testing the RAG QA System (using loaded index) ---")
    test_queries = [
         "What schemes are available for farmers in Maharashtra?",
         "What is the benefit amount for the PM Scholarship for RPF?",
         "Tell me about eligibility for Snehasanthwanam.",
         "Are there any schemes for building houses?",
         "What are the documents required for the Laadli Laxmi Scheme?",
         "What is the MyScheme portal?" # Expected to fail gracefully
     ]
    for i, q in enumerate(test_queries):
         print(f"\n--- Query #{i+1} ---")
         print(f"Q: {q}")
         answer = ask_scheme_question_rag(q)
         print(f"A: {answer}") # Also print answer to console

    print("\n" + "="*50)
    log.info(f"--- RAG QA Setup & Test Finished. Duration: {time.time() - start_rag_time:.2f} sec ---")
    return True # Indicate success


# --- Script Entry Point ---
if __name__ == "__main__":
    overall_start_time = time.time()
    log.info("Starting RAG QA script (using pre-built index)...")

    # Directly run the RAG setup and testing function
    rag_success = setup_and_run_qa_system()

    # Final Summary
    overall_end_time = time.time()
    log.info("RAG QA script finished.")
    print("\n" + "="*60)
    print("RAG QA Script Execution Summary:")
    print("="*60)
    if rag_success:
        print(f"- RAG QA Phase: Completed (Index/Data loaded from files, tests run)")
    else:
        print("- RAG QA Phase: Failed or Did Not Complete (Check logs for errors)")

    print(f"\nTotal Execution Time: {overall_end_time - overall_start_time:.2f} seconds.")
    print("="*60)


--- Testing the RAG QA System (using loaded index) ---

--- Query #1 ---
Q: What schemes are available for farmers in Maharashtra?
A: ANKY

--- Query #2 ---
Q: What is the benefit amount for the PM Scholarship for RPF?
A: Scholarships to Students with Disabilities

--- Query #3 ---
Q: Tell me about eligibility for Snehasanthwanam.
A: The applicant should be a resident of Kerala State

--- Query #4 ---
Q: Are there any schemes for building houses?
A: Storm Water Drainage

--- Query #5 ---
Q: What are the documents required for the Laadli Laxmi Scheme?
A: Laadli Laxmi Scheme

--- Query #6 ---
Q: What is the MyScheme portal?
A: A receipt should be sent to the concerned District.


RAG QA Script Execution Summary:
- RAG QA Phase: Completed (Index/Data loaded from files, tests run)

Total Execution Time: 5.34 seconds.
